In [1]:
import tweepy
import json
import pandas as pd
from perspective import PerspectiveAPI
from googleapiclient import discovery
import numpy as np
import re
import time
from tqdm.notebook import tqdm
path_files ="C:/Users/gianl/Desktop/Gi/Supsi/Vaccines_Discussion_Italy/Laura/Files/"
path_data = "C:/Users/gianl/Desktop/Gi/Supsi/Vaccines_Discussion_Italy/Laura/Tweets/"

In [2]:
jsonFile = open(path_files + 'auth.json', 'r')
config = json.load(jsonFile)
jsonFile.close()
API_KEY = config["api_key_1"]

In [6]:
def clean(x):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split())

In [3]:
df_bad = pd.read_csv(path_data + "bad_activities.csv")
df_good = pd.read_csv(path_data + "good_activities.csv")
df_suspended = pd.read_parquet(path_data + "tweets_bad_suspended.parquet", engine="fastparquet")
df_suspended_good= pd.read_parquet(path_data + "tweets_good_suspended.parquet", engine="fastparquet")
tweets = pd.read_parquet(path_data + "tweets.parquet", engine="fastparquet")

In [4]:
tweets.head()

,id,id_str,created_at,user_id,user_screen_name,user_verified,user_created_at,text,source,coordinates,...,quoted_user_created_at,hashtags,symbols,user_mentions,urls,polls,media,user_tweets_count,user_friends_count,user_followers_count
0,1221739084449832960,1221739084449832960,Mon Jan 27 10:17:51 +0000 2020,241972508,planetpaul65,False,Sun Jan 23 16:31:07 +0000 2011,La Memoria è l’unico vaccino contro l’indiffer...,"<a href=""http://twitter.com/download/android"" ...",None,...,None,"[{'text': 'GiornatadellaMemoria2020', 'indices...",[],"[{'screen_name': 'isentinelli', 'name': '@isen...",[],[],"[{'id': 1221708229090127872, 'id_str': '122170...",NaN,NaN,NaN
1,1221737155380293640,1221737155380293640,Mon Jan 27 10:10:11 +0000 2020,1179711277,Grace_1207,False,Thu Feb 14 16:37:35 +0000 2013,"Oggi sono arrivati loro, 3 femminucce e 1 masc...","<a href=""https://mobile.twitter.com"" rel=""nofo...",None,...,None,[],[],"[{'screen_name': 'LidaSezOlbia', 'name': 'Lida...",[],[],[],NaN,NaN,NaN
2,1221730009989120001,1221730009989120001,Mon Jan 27 09:41:47 +0000 2020,115121530,MarisaMinervini,False,Wed Feb 17 17:40:09 +0000 2010,L'indifferenza è più colpevole della violenza ...,"<a href=""http://twitter.com/download/android"" ...",None,...,None,[],[],"[{'screen_name': 'milio967', 'name': 'Εμιλιο ♌...",[],[],[],NaN,NaN,NaN
3,1221737654909382662,1221737654909382662,Mon Jan 27 10:12:10 +0000 2020,589884969,LecceSette,False,Fri May 25 08:09:46 +0000 2012,"Minaccia morbillo nel Salento, Asl: ""Vaccinars...","<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",None,...,None,[],[],[],"[{'url': 'https://t.co/CY0qiQFk6s', 'expanded_...",[],[],NaN,NaN,NaN
4,1221728980203724801,1221728980203724801,Mon Jan 27 09:37:42 +0000 2020,908731428124397568,arual812,False,Fri Sep 15 16:37:10 +0000 2017,"""Coltivare la Memoria è ancora oggi un vaccino...","<a href=""http://twitter.com/download/android"" ...",None,...,None,[],[],"[{'screen_name': 'Annalisa3073', 'name': 'Anna...",[],[],[],NaN,NaN,NaN


In [7]:
x = list(set(tweets["text"]))
count = 0
lst = []
for i in tqdm(x):
    lst.append(clean(i))

In [8]:
x2 = list(set(lst))
len(x2)

1499124

In [15]:
# pd.DataFrame(x2[1:50], columns=["text"]).to_csv("C:/Users/gianl/Desktop/Gi/Supsi/Vaccines_Discussion_Italy/Laura/example.csv",
#                                                line_terminator="\n", index=False, encoding="utf-8")
# esempio = pd.read_csv("C:/Users/gianl/Desktop/Gi/Supsi/Vaccines_Discussion_Italy/Laura/texts/example.csv",
#                               lineterminator="\n", low_memory=False, encoding="utf-8")
# esempio

In [61]:
print(len(x2))
count = 0
for i in x2:
    if len(i) < 30:
        count = count + 1
        print(i)
print(count)
1499124 - 69376
1500000 / 50000

1429748

# DIfferences between Clean and Dirty text

In [27]:
print(texts[4])
print("----------")
x2 = clean(texts[4])
print(x2)

Si dice che il Messia #Draghi ha fatto sentire la sua voce in #Europa, per fortuna era in videoconferenza altrimenti avrebbe menato le mani. 🙄
#governodeipeggiori #vaccini #Piazzapulita
----------


In [29]:
x = texts[4]
p = PerspectiveAPI(api_key=API_KEY)
print(p.score(x))
print(p.score(x2))

{'TOXICITY': 0.05373499}
{'TOXICITY': 0.27045983}


# Classification

In [68]:
p = PerspectiveAPI(api_key=API_KEY)
results = []
for i in tqdm(texts):
    time.sleep(1)
    try:
        result = p.score(i)
        results.append(result["TOXICITY"])
    except Exception as e:
        print("Error: ", e)

Error:  HTTP Error 400: Bad Request
Error:  HTTP Error 400: Bad Request
Error:  HTTP Error 400: Bad Request
Error:  HTTP Error 400: Bad Request
Error:  HTTP Error 400: Bad Request
Error:  HTTP Error 400: Bad Request



KeyboardInterrupt: 

In [69]:
np.mean(results)

0.10648387772184312

In [70]:
print(f"Primi 1000 tweet Good {0.03} Toxicity score")
print(f"Primi 1000 tweet Good susp {0.10} Toxicity score")
print(f"Primi 1000 tweet Bad susp {0.12} Toxicity score")
print(f"Primi 1000 tweet Bad {0.09} Toxicity score")

Primi 1000 tweet Good 0.03 Toxicity score
Primi 1000 tweet Good susp 0.1 Toxicity score
Primi 1000 tweet Bad susp 0.12 Toxicity score
Primi 1000 tweet Bad 0.09 Toxicity score


In [79]:
client = discovery.build(
  "commentanalyzer",
  "v1alpha1",
  developerKey=API_KEY,
  discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
  static_discovery=False,
)

analyze_request = {
  'comment': { 'text': "mario, guarda, sei bravo!"},
  'requestedAttributes': {'TOXICITY': {}}
}

response = client.comments().analyze(body=analyze_request).execute()
print(json.dumps(response, indent=2))

{
  "attributeScores": {
    "TOXICITY": {
      "spanScores": [
        {
          "begin": 0,
          "end": 25,
          "score": {
            "value": 0.024833864,
            "type": "PROBABILITY"
          }
        }
      ],
      "summaryScore": {
        "value": 0.024833864,
        "type": "PROBABILITY"
      }
    }
  },
  "languages": [
    "it"
  ],
  "detectedLanguages": [
    "it"
  ]
}
